#**오차역전파법**
* 가중치 매개변수의 기울기를 효율적으로 계산

* 역전파 구현하기
  * 곱셈계층
  ```
  class MulLayer:
      def __init__(self):
          self.x = None
          self.y = None

      def forward(self, x, y):
          self.x = x
          self.y = y
          out = x * y
          return out

      def backward(self, dout):
          # dout = upstream gradient
          # dx, dy = downstream gradient
          # find local gradient
          dx = dout * y
          dy = dout * x

          return dx, dy
  ```

    ```
    220.00000000000003
    2.2 110.00000000000001 200
    ```
  * 덧셈계층
    ```
    class AddLayer:
        def __init__(self):
            pass

        def forward(self, x, y):
            out = x + y
            return out

        def backward(self, dout):
            # dout = upstream gradient
            # dx, dy = downstream gradient
            # find local gradient
            dx = dout * 1
            dy = dout * 1

            return dx, dy
    ```

    ```
      715.0000000000001
      110.00000000000001 2.2 3.3000000000000003 165.0 650
    ```
  * TwoLayerNet 클래스로 구현
  
  <img src="https://drive.google.com/uc?id=1HShQ8NHL1XqpvOJmzmWip7HztR2gEdk1" width="300" hegith="300"/>
    
    * y_pred의 초기값에서 반복문을 돌면서 점차 y_pred값이 y값과 가까워지는 것을 확인할 수 있다.
  
    ```python
        #w1: 1번째 층의 가중치
        #w2: 2번째 층의 가중치

        N, Din, H, Dout = 64, 1000, 100, 10
        x, y = randn(N, Din), randn(N, Dout)
        w1, w2 = randn(Din, H), randn(H, Dout)

        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        y_pred_first = -1
        for t in range(2000):

            # h1 = x * w1
            h1 = x.dot(w1)
            # h2 <- sigmoid
            h2 = sigmoid(h1)

            # y_pred = h2 * w2
            y_pred = h2.dot(w2)
            if t == 0:
                y_pred_first = y_pred

            # loss (y-y')^2
            loss = np.square(y_pred-y).sum()

            # backprop
            #dloss/dy_pred = -2(y-y_pred)
            dy_pred = 2*(y_pred-y)

            # dw2 -> dy_pred/dw2 * dy_pred
            # local h2.T
            # dh2 -> d(h2 * w2)/dh2 * dy_pred
            # local w2
            dw2 = h2.T.dot(dy_pred)
            # dh2
            dh2 = dy_pred.dot(w2.T)
            # sigmoid back propagation
            dh = dh2 * h2 * (1-h2)

            # dw1
            # h1 = x * w1
            # (dh1/dw1) * dh  (upstream gradient)
            # dh1 / dw1 = d (x*w1) /dw1 = x
            dw1 = x.T.dot(dh)

            w1 -= 1e-4 * dw1
            w2 -= 1e-4 * dw2

            print(loss)

        print('x: ', x)
        print('y: ', y)
        print('y_pred_first: ', y_pred_first)
        print('y_pred: ', y_pred)
    ```
    * loss값이 점점 줄어드는 것을 확인할 수 있다.

      ```python
          27916.85048580332
          19151.566974793648
          15279.771431372079
          ....
          255.2662699910044
          254.11898564834232
          248.50093736150404
          247.40256260387298
          .....
          10.755388781712387
          10.735353119120752
          10.715357494556331
        ```
  * params : 신경망의 매개변수를 보관하는 딕셔너리 변수.
        * params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향.
        * params['W2']은 2번째 층의 가중치, params['b2']은 2번째 층의 편향.
  * layers : 신경망의 계층을 보관하는 순서가 있는 딕셔너리 변수
        * layers['Affine1'], layers['Relu1'], layers['Affine2']와 같이 각 계층을 순서대로 유지
  * lastLayer : 신경망의 마지막 계층(여기서는 SoftmaxWithLoss)

    ```python
  class Affine:
      def __init__(self, W, b):
          self.W = W
          self.b = b
          self.x = None
          self.dW = None #input_size, hidden_size
          self.db = None #scalar - single value

      def forward(self, x):
          ## w*x + b
          self.x = x
          out = np.dot(x, self.W) + self.b

          return out

      def backward(self, dout):
          ##
          # dout - upstream gradient
          # dx - downsteram gradient
          ##
          self.dw = self.x.T.dot(dout)
          self.db = dout.sum()
          dx = dout.T.dot(self.x)

          return dx
    ```
    ```python
    class Relu:
        def __init__(self):
            self.mask = None

        def forward(self, x):
            # relu(1) = 1 <- max(0,1)
            # relu(-1) = 9 <- max(0, -1)
            self.mask = (x <= 0)
            out = x.copy()
            out[self.mask] = 0

            return out

        def backward(self, dout):
            # dout - upstream gradient
            # dx - downsteram gradient
            dout[self.mask] = 0
            dx = dout

            return dx
    ```
    ```python
        def gradient(self, x, t):
            # 순전파
            self.loss(x, t)
            # 역전파 구현
            dout = 1
            d = self.lastLayer.backward(dout)
            d = self.layers['Affine2'].backward(d)
            d = self.layers['Relu1'].backward(d)
            d = self.layers['Affine1'].backward(d)

            # 결과 저장
            grads = {}
            grads['W1'] = self.layers['Affine1'].dW
            grads['b1'] = self.layers['Affine1'].db
            grads['W2'] = self.layers['Affine2'].dW
            grads['b2'] = self.layers['Affine2'].db
            return grads

    ```
  * 신경망의 계층을 순서가 있는 딕셔너리에서 보관한다.
      * 순전파 때는 추가한 순서대로 각 계층의 forward()를 호출하기만 하면 된다.
      * 역전파 때는 계층을 반대 순서로 호출하기만 하면 된다.
  * 신경망의 구성 요소를 모듈화하여 계층으로 구현했기 때문에 구축이 쉬워진다.

#**Optimization**
어떤 loss function이 주어지면, 이 loss를 최소화하기위한 w 값을 찾는것.

<img src="https://drive.google.com/uc?id=1fIb8_Kalhz2BLJ8BgctOGCFWzDoS5CxO" width="200" height="30">

- RandomSearch: 시간 오래걸림
- Slope 따라가기: 미분의 개념

  <img src="https://drive.google.com/uc?id=10WSXjOGVBJB2JLftpOZur9XbZZun-oH1" width="250" height="60">
  
  - loss에 대한 gradient descent 구하기.
    - loss를 최적화하는 w_1, w_2 지점을 찾기.
    - 각 지점에서 gradient를 구하고, 매 순간마다 업데이트를 하여 중간지점으로 수렴된다.
  - Batch Gradient Descent
    - 전체 데이터셋 N에 대해서 최적값을 구한다.
  - Stochastic Gradient Descent
    - 전체 데이터 셋 N이 클 경우, computation이 어렵기 때문에 일부 데이터만 샘플링하여 gradient를 구한다.
    <img src="https://drive.google.com/uc?id=1D-8ISMssZ4k0H4bU9lbqaFIUswUK4wuc" width="550" height="250">
    - 구현
      ```
      class SGD:
        def __init__(self, lr=0.01):
            self.lr = lr

        def update(self, params, grads):
            for key in params.keys():
                #params[key]:weight
                #grads[key]:gradient
                params[key] -= self.lr * grads[key]
      ```

  - 단, 미니배치를 뽑아서 사용할때 문제점 발생 - 추가적인 장치를 더하여 SGD의 단점을 보완함.
  - 한계
    - loss 값이 각 방향마다 변화가 빠르거나 느릴경우, slow progress, shallow dimension, jitter 패턴으로 나타날 수 있음.
    - local minimum (더 낮은 loss 값이 있음에도 못찾음)에 빠지거나, saddle point(미분값이 0이되어 더이상 움직이지 않음)가 발생.
    - gradient가 noisy될 가능성이 있음.

  - SGD Momentum (관성)
    - velocity: 기존에 업데이트된 방향을 트래킹하여 더함.
    - SGD보다 noisy도 적고, loss 최적값에 빨리 도달함.
    <img src="https://drive.google.com/uc?id=1lLdFHNmO7XpaRjBTQOWp9R-zlL2wF4eH" width="550" height="250">

    - 구현
    ```
    class Momentum:
        def __init__(self, lr=0.01, momentum=0.9):
            self.lr = lr
            self.momentum = momentum
            self.v = None

        def update(self, params, grads):
            if self.v is None:
                self.v = {} #v initialization
                for key in params.keys():
                    self.v[key] = np.zeros_like(params[key])

            for key in params.keys():
                self.v[key] = self.momentum * self.v[key] + grads[key]
                self.params[key] -= self.lr * self.v[key]
    ```

    - Nesterov Momentum
      - look ahead: velocity를 더할 것을 알고 있기 때문에 weight에 velocity를 미리 더해주고 gradient를 구한다.
      - 즉, gradient를 선반영하고 구하기.

        <img src="https://drive.google.com/uc?id=15H5A2uZMscEXH-CQG62wcT3nInSv5b9B" width="250" height="50">

- AdaGrad
    - 너무 많은 업데이트가 되어있는 것들에는 normalization을 해주는 방식의 optimizer.
    - historical sum of squares in each dimension.
    - steep한 direction은 완만하게, flat한 direction은 경사를 높임.
    - 구현
    ```
    class AdaGrad:
        def __init__(self, lr=0.01):
            self.lr = lr
            self.h = None

        def update(self, params, grads):
            #init grad_squared
            if self.h is None:
                self.h = {}
                for key, val in params.items():
                    self.h[key] = np.zeros_like(val)
            #update weights
            for key in params.keys():
                self.h[key] += grads[key] * grads[key]
                params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)
    ```

- RMSProp
  - AdaGrad는 계속 더해지기 때문에 update가 안되는 문제점이 있기 있음.
  - 따라서, decay_rate를 통해 일정비율로 update 될수 있도록 normalization 한다.
  - 구현
  ```
    class RMSprop:
      def __init__(self, lr=0.01, decay_rate=0.99):
          self.lr = lr
          self.decay_rate = decay_rate
          self.h = None

      def update(self, params, grads):
          if self.h is None:
              self.h = {}
              for key in params.keys():
                  self.h[key] = np.zeros_like(params[key])

          for key in params.keys():
              self.h[key] = self.decay_rate * self.h[key] + (1 - self.decay_rate) * grads[key]
              params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)
  ```

- Adam
  - RMSProp + Momentum을 둘다 사용
  - 구현
  ```
  class Adam:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None

    def update(self, params, grads):

        if self.v is None:
            self.v = {}
            self.m = {}
            for key in params.keys():
                self.v[key] = np.zeros_like(params[key])
                self.m[key] = np.zeros_like(params[key])

        self.iter += 1

        for key in params.keys():
            self.v[key] = self.beta1 * self.v[key] + (1 - self.beta1) * grads[key]
            self.m[key] = self.beta2 * self.m[key] + (1 - self.beta2) * grads[key] * grads[key]

            moment1_unbias = self.v[key] / (1 - self.beta1**self.iter)
            moment2_unbias = self.m[key] / (1 - self.beta2**self.iter)
            params[key] -= self.lr * moment1_unbias / (moment2_unbias.sqrt() + 1e-7)
  ```
- 구현결과

  <img src="https://drive.google.com/uc?id=1eBDKuRRFXIMkxIGwLub7UgXSJ3rE2S2O" width="650" height="350">
- First-Order Optimization:1차 미분
- Second-Order Optimization:2차 미분 (Hessian matrix가 필요함)
  - Hessian matrix는 비용이 많이 듬.
  - 잘 사용하지 않음.
- Convex Functions
  - local minimum이 없는 함수.
  - 그래프의 두점을 잇는 선보다 그래프의 선이 항상 낮아야함.
  - 만일 높다면, convex function이 아님 -> local minimum/saddle point가 존재함.

#CNN
- 이미지를 vectorize를 통해 이미지 변환이 필요한데, spatial한 구조의 이미지에 대해 structure한 정보가 훼손되기 때문에 이를 보존하기 위한 방법.
- Convolution layers/pooling layers/Normalization을 사용한다.
- Convolution layer
  - 최대한 spatial structure를 보존함.
    1. 필터를 이미지에 적용함.(dot products로 계산)
    2. activation map을 형성.
    3. 각 필터 하나당 activation map이 생성됨.
    <img src="https://drive.google.com/uc?id=17kEQr3drXCav-N7FJGCFpsisRNm1LdNA" width="550" height="250">

  - Stcking Convolutions
    - convolution layer를 stack으로 쌓기.
    - 계속 stack으로 쌓아도 non linear한 layer이기 때문에 layer하나 쓰는 것과 차이점이 없음
    - 따라서, 중간에 non linear 값을 넣어줘야함.(ReLU)
    - 초기 layer는 단순패턴과 구조를 찾고, 뒤로 갈수록 구체적인 패턴을 찾게 된다.
  
  - for문으로 convolution 구현

    - Input이 일정한 X차원에서 filter 적용함
    - for문이 3중 for문으로 계산이 느림
    ```
    for i in range(output.shape[0]):
      for j in range(output.shape[1]):
        ## implement convolution
        for channel in range(x.shape[0]):
            output[i, j] += np.sum(x[channel, i:i+fh, j:j+fw] * f[channel])
    ```
    - im2col 구현
      - 필터와 연산하는 부분이 정해져있으니, 그 부분을 각각 뺴내어 필터와 행렬연산을 하도록 데이터 형상을 조정
      - 입력데이터를 가중치 계산하기 편하도록 4차원 이미지인 입력데이터와 filter를 모두 2차원 행렬로 전개하는 함수
  ```
    def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
        N, C, H, W = input_data.shape
        out_h = int((H + 2*pad - filter_h) / stride + 1)
        out_w = int((W + 2*pad - filter_w) / stride + 1)

        img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
        col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

        ## Col 완성
        for h in range(filter_h):
            h_max = h + stride * out_h
            for w in range(filter_w):
                w_max = w + stride * out_w
                col[:, :, h, w, :, :] = img[:, h:h_max, w:w_max]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
        return col
   ```
    - col2im 구현
     ```
      def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
        N, C, H, W = input_shape
        out_h = (H + 2*pad - filter_h) // stride + 1
        out_w = (W + 2*pad - filter_w) // stride + 1
        col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)
        
        img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
        for y in range(filter_h):
            y_max = y + stride*out_h
            for x in range(filter_w):
                x_max = x + stride*out_w
                img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]
              
      return img[:, :, pad:H + pad, pad:W + pad]
     ```
    - forward
    ```
      def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)

        # 입력 데이터와 필터를 2차원 배열로 전개하고 내적한다.
        self.x = x
        self.col_w = self.w.reshape(FN, C*HF*FW)
        self.col = im2col(x, filter_h=FH, filter_w=FW, stride=self.stride, pad=self.pad)

        out = self.col.dot(self.col_w.T) + self.b
        out = out.reshape(N, out_h, out_w, FN)
        out = out.transpose(0, 3, 1, 2)

        # reshape에서 -1 : 원소 개수에 맞춰 적절하게 묶어줌.
        # transpose : 다차원 배열의 축 순서를 바꿔줌(N,H,W,C) -> (N,C,H,W)
        return out
     ```
    - backward
    ```
      def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0, 2, 3, 1)
        dout = dout.reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dw = np.dot(self.col.T, dout)
        self.dw = self.dw.transpose(1, 0)
        self.dw = self.dw.reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_w.T)
        dx = col2im(dcol, filter_h=FH, filter_w=FW, stride=self.stride, pad=self.pad)

        return dx
    ```

  - Spatial Dimension

   필터를 적용하면 output값이 줄어들기 때문에 input값 주변에 0값의 padding을 주어서 input값과 output값을 같게 만든다.

  - Receptive Fields
    - 이미지 인식시, Receptive Fields가 클수록 더 큰 영역에서 이미지를 인식할 수 있도록 함.
    - 단, 이미지가 커지면, 더 많은 layer들이 필요로 하게되는 문제가 발생한다.
    - downsample, 이미지를 줄일 수 있도록 함.
      - Strided Convolution: sliding window시, stride를 넣어서 볼륨사이즈 줄이기

- Pooling Layer
  - 각 이미지별 max값만 pooling하여 이미지를 줄일 수 있다.
  <img src="https://drive.google.com/uc?id=1ODo4ZnoKzdb4mcWFZ_Gr3Fj51w1Upd6y" width="550" height="250">
  - 구현
  ```python
    class Pooling:
      def __init__(self, pool_h, pool_w, stride=1, pad=0):
          self.pool_h = pool_h
          self.pool_w = pool_w
          self.stride = stride
          self.pad = pad

      def forward(self, x):
          N, C, H, W = x.shape
          out_h = int(1 + (H - self.pool_h) / self.stride)
          out_w = int(1 + (W - self.pool_w) / self.stride)

          #im2col이용하여 구현
          col
          = im2col(x, filter_h=self.pool_h, filter_w=self.pool_w,stride=self.stride, pad=self.pad)
          # 전개
          col = col.reshape(-1, self.pool_h * self.pool_w)
          out = np.max(col, axis=1)
      
      return out
  ```

#**RNN**
- 다양한 Task가 존재
  - Image Captioning: input이 image일때, output을 text로 출력
    - one to many: 단어 하나하나가 output
  - Video Classification: image를 하나의 비디오로 출력
    - many to one: 여러 image input에 대한 하나의 라벨(비디오)
  - Machine Translation
    - many to many: 문장을 다른 언어로 번역
  - Per-frame video Classification:
    - many to many: 연속된 image에 대한 연속된 라벨(비디오)

- Vanila RNN
  - RNN중간에 internal state를 갖고 매순간 계속 반복적으로 업데이트 된다.
  - 수식
    - w는 모든 스텝에 대해서 동일하게 적용된다.
    <img src="https://drive.google.com/uc?id=1Ebm7vtDESijDhBWIUQ0FLhQM1uF6Ksez" width="550" height="250">
  
  - loss 측정
    - many to many: 모든 hidden state에 대해서 y를 뽑아내고, 이에 대한 loss를 측정할 수 있다.
    - many to one: 중간에 output 존재하지 않고, 끝에서만 output 출력하여 이에 대한 loss 한번만 측정
    - one to many: 각 hidden state에 대해 y 출력하고 이에 대한 각각의 loss 측정
  
  - 구현
    ```python
        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

    ```
    ```
        def forward(self, x, h_prev):
        Wx, Wh, b = self.params

        ## RNN implementation
        out = np.dot(h_prev, Wh.T) + np.dot(x, Wx.T)
        h_next = np.tanh(out)
        ##

        self.cache = (x, h_prev, h_next)
        return h_next
    ```
    - 결과: Perplexity 감소하는 것을 확인할 수 있다.

      <img src="https://drive.google.com/uc?id=1g-gQXhT1GwNIIbqd-jmOLc0gTpj_0Kbh" width="250" height="150">

- Language Modeling
  - ex) 주어진 character에서 다음에 올 글자를 찾기 (hello)
    - h -> e -> l -> l -> 0
    - 각 글자마다 hidden state를 생성
    - x: 각 글자의 벡터값, h: 각 글자마다 hidden layer
    - 각 글자의 벡터값은 one-hot vector로 표현 (하나만 1로 표현하고 나머지는 0으로 표현)
    - w*h*y를 곱해서 output에 대해 loss를 통해 타겟 글자를 예측하도록 함
    - embedding layer: 각 글자에 대한 벡터를 어떤 특정한 값을 embedding 함
    (input layer -> embedding layer -> hidden layer)
    <img src="https://drive.google.com/uc?id=12o_ExaKWuj8IwCE1zr0ccrkA0LPR_Tfq" width="550" height="250">

- Truncated Backpropagation Through Time
  - output이 여러개 존재할때, 모든 각 output에 대해 backpropagation시, 마지막 output은 모든 sequence를 지나야 한다.
  - 이때, 많은 memory와 시간이 필요로 하기 때문에 일부시점만 truncate하여서 backpropagation을 진행한다.
  - forward/backward를 chunk로 쪼개서 진행서 메모리를 적게사용하도록 함.

- Interpretable Hidden Units
  - output을 생성하기 전에 input sequence 중, 가장 interpret하기 쉬운것이 무엇인지 확인함
  - 특정 한 cell부분에 activation이 강하게 나타나는 부분을 확인할 수 있다.

- Image Captioning
  - Image에 대한 caption을 생성함.
  - CNN + RNN 으로 이루어져있다.
  - Transfer Learning: 이미 훈련된 CNN을 갖고 image를 넣는다.
  - 해당 이미지를 갖고 각각의 hidden state를 생성하여, 이미지에 대한 caption을 생성
    <img src="https://drive.google.com/uc?id=1wo2qb4uQ2mLgJQ2Kv_qKj1vuf5FcAaIf" width="550" height="250">
  
- 자연어와 단어의 분산표현
  - 단어들은 실수의 값을 가지고 있지 않음
  - Thesaurus: 뜻이 비슷한 단어를 그룹으로 분류하고 단어 사이의 상위와 하위관계를 트리로 체계화
    - ex) Car = auto automobile machine motorcar
    - 문제점: 1)사람들이 직접 개입하여 수작업 함. 2)계속 매뉴얼리 업데이트 해야함.
  - 이미지에서 픽셀이 이미지의 최소단위라면, 단어는 말뭉치의 최소단위다. 자연어처리의 첫 단추는 단어의 벡터표현이다.
  - 희소표현 (one-hot vector encoding)
    - 예) You say goodbye and I say hello
      - Unique words: You, say, goodbye, and, I, hello
      <img src="https://drive.google.com/uc?id=1k6KqQpyXykIV9LJakPso41AEnjxTCIjC" width="550" height="250">
    
    - 문제점
      - 벡터의 크기가 지나치게 커서 dimension표현이 커지는 비효율 발생.
      - vocab안에 없는 단어가 발생할 수 있음.
      - 벡터가 단어의 의미를 표현하지 않는다. 각 글자간 거리가 다 같아서 관련성을 표현하기 어려움.

    - 해결
      - 동시발생행렬
        - 단어 근처에 단어에 대한 연관성을 매트릭스로 표현.
        - 코사인 유사도: 두벡터가 가리키는 방향이 얼마나 비슷한가를 나타내는 양.
        - 문제점:
          - 정관사/부사 등, 단어에 함께 쓰이기 때문에 단순 카운팅으로는 단어간 깊은 연관성을 표현하는데 어려움이 있다.
        - 따라서, 동시발생행렬에 총합을 나눠서 확률분포를 나타냄.(Pointwise Mutual Information)
        - PPMI 행렬: 단어의 빈도수 만큼 나눠준 값을 갖는 새로운 행렬.

    - 구현
      - 동시발생행렬 구현
      ```python
        def create_co_matrix(corpus, vocab_size, window_size=1):
          corpus_size = len(corpus)
          co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

          for idx, word_id in enumerate(corpus):
              for i in range(1, window_size + 1):
                  left_idx = idx - i
                  right_idx = idx + i

                  if left_idx >= 0:
                  ###
                      previous_word_id = corpus[left_idx]
                      co_matrix[word_id, previous_word_id] += 1
                      
                  if right_idx < corpus_size:
                  ###
                      next_word_id = corpus[right_idx]
                      co_matrix[word_id, next_word_id] += 1

        return co_matrix
    ```
        - 결과

          <img src="https://drive.google.com/uc?id=1UEgzBB-ijs0tZ1lXDev5S8Jtqnl645Dz" width="200" height="200"> <img src="https://drive.google.com/uc?id=1Vaq6qwZORDgJ4vQc0_-3zyoYnXuGb7UG" width="300" height="200">

      - 코사인유사도
      ```python
        def cos_similarity(x, y, eps=1e-8):
          x_sqrt_sum = np.sqrt(np.sum(x*x))
          y_sqrt_sum = np.sqrt(np.sum(y*y))
          out = np.dot(x,y) / (x_sqrt_sum * y_sqrt_sum)

        return out
      ```
      ```
        ##
        c0 = C[word_to_id['you']]  # "you"의 단어 벡터
        c1 = C[word_to_id['i']]    # "i"의 단어 벡터
        print(cos_similarity(c0, c1))
        ##
        ##
        c0 = C[word_to_id['you']]
        c1 = C[word_to_id['goodbye']]
        print(cos_similarity(c0, c1))
        ##
        ##
        c0 = C[word_to_id['i']]
        c1 = C[word_to_id['goodbye']]
        print(cos_similarity(c0, c1))
        ##
      ```
        - 결과

          <img src="https://drive.google.com/uc?id=1hf5v0Sn4LBXbYPY4B1OkrtAPbBaPlbvO" width="300" height="150">

      - PPMI
      ```python
          def ppmi(C, verbose=False, eps = 1e-8):
            M = np.zeros_like(C, dtype=np.float32)
            N = np.sum(C)
            S = np.sum(C, axis=0)
            total = C.shape[0] * C.shape[1]
            cnt = 0

            for i in range(C.shape[0]):
                for j in range(C.shape[1]):
                    pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
                    M[i, j] = max(0, pmi)

                    if verbose:
                        cnt += 1
                        if cnt % (total//100 + 1) == 0:
                            print('%.1f%% 완료' % (100*cnt/total))
            return M
      ```
        - 결과

          <img src="https://drive.google.com/uc?id=1YURKuB3o_QRfX4yMjqE6cKRtDSTVzQ5j" width="250" height="350">

           <img src="https://drive.google.com/uc?id=1HyCUPWB1ofvy7WGUorUrqGTmeITlERye" width="600" height="350">

- word2vec
  - 데이터에서 Neural Network를 통해서 단어들의 representation을 학습
  - Count-Based: 단어, 문맥 출현 횟수를 센다. (동시발생행렬)
  - Prediction-Based: 단어에서 문맥/ 문맥에서 단어를 예측하는 방법 (word2vec)
  - CBOW 모델
    - 맥락의 벡터값에서 타깃값을 예측한다.
    - Win -> W out을 예측
    - 각각 두개의 단어표현에 대한 평균을 구해서 또 다른 Wout 매트릭스에 대입
    <img src="https://drive.google.com/uc?id=19pUFQWjKtac_UpFvfiJLZ6sv1uVvC5vS" width="450" height="350">

    - 구현
    ```python
        class MatMul:
          def __init__(self, W):
              self.params = [W]
              self.grads = [np.zeros_like(W)]
              self.x = None

          def forward(self, x):
              W, = self.params
              out = np.dot(x, W)
              self.x = x
              return out

          def backward(self, dout):
              W, = self.params
              dx = np.dot(dout, W.T)
              dW = np.dot(self.x.T, dout)
              self.grads[0][...] = dW
              return dx

        # 샘플 맥락 데이터
        c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
        c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

        # 가중치 초기화
        W_in = np.random.randn(7, 3)
        W_out = np.random.randn(3, 7)

        # 계층 생성
        first_layer = MatMul(W_in)
        second_layer = MatMul(W_out)

        # 순전파
        out_1 = first_layer.forward(c0)
        out_2 = second_layer.forward(c1)
        out = out_1 + out_2
        out = out*0.5
        final_out = second_layer.forward(out)
    ```

  - Skip-gram 모델
    - 중앙의 단어로부터 맥락을 추측
    - 하나 input을 가지고 2개의 output을 예측한다.
    <img src="https://drive.google.com/uc?id=1ghaTz89AD0BYgmzzd6RzXfqdb0W1lPuS" width="450" height="350">

    - 구현
    ```python
        # 샘플 맥락 데이터
        c0 = np.array([[0, 1, 0, 0, 0, 0, 0]])
        c1 = np.array([[1, 0, 0, 0, 0, 0, 0]])
        c2 = np.array([[0, 0, 1, 0, 0, 0, 0]])

        # 가중치 초기화
        W_in = np.random.randn(7, 3)
        W_out = np.random.randn(3, 7)

        # 계층 생성
        first_layer = MatMul(W_in)
        second_layer = MatMul(W_out)

        # 순전파
        out = first_layer.forward(c0)
        final = second_layer.forward(out)
    ```



    

        







